#Code implementation of the paper "To Act or React?"
https://arxiv.org/pdf/1906.11932v1.pdf

Connect: ahmedrashidqazi@gmail.com

###Libraries

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import sklearn
from google.cloud import bigquery
import re

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
client = bigquery.Client('myproject123-319010')

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#Data
##Time Frame - 01/2015 - 10/2018

###DA = Most Active 3 thousand Subreddits based on the average number of posts per month

In [5]:
DA = []
DA = pd.read_csv("/content/gdrive/MyDrive/Most Active 3000 Subreddit Names.csv")
DA

,subreddit,a
0,AskReddit,210116.4
1,AutoNewspaper,183784.6
2,RocketLeagueExchange,146558.1
3,CircleofTrust,142087.7
4,StocksInFocus,128409.5
...,...,...
2995,Rainmeter,674.0
2996,funwithwords,673.4
2997,Ooer,673.1
2998,u_b206050de3,673.0


###DB = 38 Quarantined/Banned Subreddits

In [6]:
DB = ['Braincels',
 'TheRedPill',
 'Ice_Poseidon',
 'CringeAnarchy',
 'TimeToGo',
 'Ice_Poseidon2',
 'DebateAltRight',
 'OffensiveMemes',
 'thinspocommunity',
 'ZOG',
 'EDFood',
 'spacedicks',
 'whitebeauty',
 'Clown__World',
 'Clownworldwar',
 'FULLCOMMUNISM',
 'GentilesUnited',
 'NorthwestFront',
 'okbuddyanarchy',
 'fragilejewishredditor',
 'Identitarians',
 'niggas',
 'holocaust',
 'tha_pit_pit',
 'Mayo_town',
 'CringeChaos',
 'kangznsheeit',
 'cursedx100images',
 'SubforWhitePeopleOnly',
 'cursedx3images',
 'The_Donald',
 'ChapoTrapHouse',
 'AmericanJewishPower',
 'honkler',
 'WatchPeopleDie',
 'Physical_Removal',
 'altright',
 'Incels'] 

####The code below was used to extract the above 36 subreddits

In [7]:
# DB = []

# query = """ SELECT subreddit FROM fh-bigquery.reddit_posts.full_corpus_201512  WHERE STRING(TIMESTAMP_SECONDS(created_utc)) LIKE '%2015%' AND quarantine = true """ 
# x = client.query(query).to_dataframe()
# if len(x) > 0: 
#   x = x['subreddit']
#   print(x.unique())
#   for k in x.unique():
#     if k not in DB:
#       DB.append(k)

# query = """ SELECT subreddit FROM fh-bigquery.reddit_posts.2015_12  WHERE quarantine = true """ 
# x = client.query(query).to_dataframe()
# if len(x) > 0: 
#   x = x['subreddit']
#   print(x.unique())
#   for k in x.unique():
#     if k not in DB:
#       DB.append(k)

# years = ['2016_01', '2017_01', '2018_01', '2019_01']

# years = ['2018_08', '2019_01']
# for i in years:
#   year = i
#   for j in range(0, 12):
#     query = """ SELECT subreddit FROM fh-bigquery.reddit_posts.""" + year + """ WHERE quarantine = true """ 
#     print(query)
#     x = client.query(query).to_dataframe()
#     print("The query for the month", year)
#     if len(x) > 0: 
#       x = x['subreddit']
#       print(x.unique())
#       for k in x.unique():
#         if k not in DB:
#           DB.append(k)
#     else:
#       print("Empty Dataframe")
  
#     print("\n\n")
    
#     month = str(int(year[-2:]) + 1)
#     if int(month) < 10:
#       month = '0'+month
#     year = year[0:-2]+month
#     if year == "2019_09" or year == "2018_13":
#       break


# DB.append("WatchPeopleDie")
# DB.append("Physical_Removal")

In [8]:
print("Total number of banned/quarantined subreddits", len(DB))
DB

Total number of banned/quarantined subreddits 38


['Braincels',
 'TheRedPill',
 'Ice_Poseidon',
 'CringeAnarchy',
 'TimeToGo',
 'Ice_Poseidon2',
 'DebateAltRight',
 'OffensiveMemes',
 'thinspocommunity',
 'ZOG',
 'EDFood',
 'spacedicks',
 'whitebeauty',
 'Clown__World',
 'Clownworldwar',
 'FULLCOMMUNISM',
 'GentilesUnited',
 'NorthwestFront',
 'okbuddyanarchy',
 'fragilejewishredditor',
 'Identitarians',
 'niggas',
 'holocaust',
 'tha_pit_pit',
 'Mayo_town',
 'CringeChaos',
 'kangznsheeit',
 'cursedx100images',
 'SubforWhitePeopleOnly',
 'cursedx3images',
 'The_Donald',
 'ChapoTrapHouse',
 'AmericanJewishPower',
 'honkler',
 'WatchPeopleDie',
 'Physical_Removal',
 'altright',
 'Incels']

###DH = 117 Hateful Subreddits

In [ ]:
DH = ['news',
 'conspiracy',
 'kotakuinaction',
 'pics',
 'legaladvice',
 'drama',
 'politics',
 'subredditdrama',
 'askreddit',
 'videos',
 'movies',
 'announcements',
 'fatpeoplehate',
 'gaming',
 'games',
 'christianity',
 'worldnews',
 'pcgaming',
 'livestreamfail',
 'conservative',
 'bitcoin',
 'television',
 'libertarian',
 'europe',
 'topmindsofreddit',
 'subredditoftheday',
 'socialism',
 'pussypassdenied',
 'morbidquestions',
 'jordanpeterson',
 'greatawakening',
 'blackpeopletwitter',
 'aww',
 'wallstreetbets',
 'unpopularopinion',
 'uncensorednews',
 'outoftheloop',
 'islam',
 'askhistorians',
 'publicfreakout',
 'european',
 'subredditcancer',
 'mensrights',
 'h3h3productions',
 'enoughtrumpspam',
 'all',
 'tumblrinaction',
 'pussypass',
 'metacanada',
 'latestagecapitalism',
 'imgoingtohellforthis',
 'canada',
 'asktrumpsupporters',
 'againsthatesubreddits',
 '4chan',
 'thenewright',
 'sjwhate',
 'milliondollarextreme',
 'gendercritical',
 'frenworld',
 'easternsunrising',
 'catholicism',
 'anarcho_capitalism',
 'iama',
 'relationships',
 'overwatch',
 'adviceanimals',
 'ps4',
 'leagueoflegends',
 'hiphopheads',
 'funny',
 'rickandmorty',
 'parenting',
 'nfl',
 'justneckbeardthings',
 'gamingcirclejerk',
 'todayilearned',
 'sandersforpresident',
 'nsfw_gif',
 'neoliberal',
 'mapporn',
 'foodporn',
 'food',
 'amitheasshole',
 'soccer',
 'sex',
 'punchablefaces',
 'makeupaddiction',
 'magictcg',
 'houston',
 'hearthstone',
 'dnd',
 'battlefield',
 'askmen',
 'whiterights',
 'publichealthwatch',
 'europeannationalism',
 'israel',
 'coontown',
 'chadright',
 'truecels',
 'mgtow',
 'weekendgunnit',
 'mr_trump',
 'the_donalds',
 'worldpolitics',
 'nationalsocialism',
 'aznidentity',
 'mongoloidsupremacy',
 'the_adolf',
 't_d',
 'samandtolki',
 'shitpoliticssays',
 'rightwinglgbt',
 'incelabode',
 'europeans',
 'commonfilth']

####Following code was used to extract the above 117 hateful subreddits

#####Extracting the text, title, score and number of comments from the subreddit r/AgainstHateSubreddits

In [9]:
# text = pd.DataFrame([], columns = ['selftext', 'title'])

# query = """ SELECT selftext, title, score, num_comments FROM fh-bigquery.reddit_posts.full_corpus_201512  WHERE STRING(TIMESTAMP_SECONDS(created_utc)) LIKE '%2015%' AND (subreddit = 'AgainstHateSubreddits') """ 
# x = client.query(query).to_dataframe()
# if len(x) > 0: 
#   text = pd.concat([text,x])

# query = """ SELECT selftext, title, score, num_comments FROM fh-bigquery.reddit_posts.2015_12  WHERE subreddit = 'AgainstHateSubreddits' """ 
# x = client.query(query).to_dataframe()
# if len(x) > 0: 
#   text = pd.concat([text,x])

# years = ['2016_01', '2017_01', '2018_01', '2019_01']

# for i in years:
#   year = i
#   for j in range(0, 12):
#     query = """ SELECT selftext, title, score, num_comments FROM fh-bigquery.reddit_posts.""" + year + """ WHERE subreddit = 'AgainstHateSubreddits' """ 
#     print(query)
#     x = client.query(query).to_dataframe()
#     print("The query for the month", year)
#     if len(x) > 0: 
#       print(x)
#       text = pd.concat([text,x])
#     else:
#       print("Empty Dataframe")
  
#     print("\n\n")
    
#     month = str(int(year[-2:]) + 1)
#     if int(month) < 10:
#       month = '0'+month
#     year = year[0:-2]+month
#     if year == "2019_09" or year == "2018_13":
#       break

# text.to_csv("/content/gdrive/MyDrive/AgainstHateSubredditsText.csv", index = False)

#####Extracting the text, title, score and number of comments from the subreddit r/SubredditDrama

In [10]:
# text = pd.DataFrame([], columns = ['selftext', 'title'])

# query = """ SELECT selftext, title, score, num_comments FROM fh-bigquery.reddit_posts.full_corpus_201512  WHERE STRING(TIMESTAMP_SECONDS(created_utc)) LIKE '%2015%' AND (subreddit = 'SubredditDrama') """ 
# x = client.query(query).to_dataframe()
# if len(x) > 0: 
#   text = pd.concat([text,x])

# query = """ SELECT selftext, title, score, num_comments FROM fh-bigquery.reddit_posts.2015_12  WHERE subreddit = 'SubredditDrama' """ 
# x = client.query(query).to_dataframe()
# if len(x) > 0: 
#   text = pd.concat([text,x])

# years = ['2016_01', '2017_01', '2018_01', '2019_01']

# for i in years:
#   year = i
#   for j in range(0, 12):
#     query = """ SELECT selftext, title, score, num_comments FROM fh-bigquery.reddit_posts.""" + year + """ WHERE subreddit = 'SubredditDrama' """ 
#     print(query)
#     x = client.query(query).to_dataframe()
#     print("The query for the month", year)
#     if len(x) > 0: 
#       print(x)
#       text = pd.concat([text,x])
#     else:
#       print("Empty Dataframe")
  
#     print("\n\n")
    
#     month = str(int(year[-2:]) + 1)
#     if int(month) < 10:
#       month = '0'+month
#     year = year[0:-2]+month
#     if year == "2019_09" or year == "2018_13":
#       break

# text.to_csv("/content/gdrive/MyDrive/SubredditDramaText.csv", index = False)

#####Analysing the subreddit to extract names of hateful subreddits from AgainstHateSubreddits

In [11]:
AHS = pd.read_csv("/content/gdrive/MyDrive/AgainstHateSubredditsText.csv")
AHS

,selftext,title,score,num_comments
0,NaN,/r/WorldPolitics now linking ONLY to anti-immi...,74.0,30.0
1,NaN,A hate subreddit brigade.,45.0,16.0
2,NaN,Action Taken by mods - Claims that /r/Europe i...,30.0,7.0
3,NaN,"""We need to start a trend on twatter #RemoveKe...",10.0,3.0
4,NaN,Test post please ignore,1.0,1.0
...,...,...,...,...
22973,[removed],What the actual f*ck is this sub,1.0,0.0
22974,[removed],Some thoughts I've got from being on this sub,1.0,0.0
22975,[removed],META: Should direct reddit links be banned on ...,18.0,8.0
22976,[removed],Pro-China Protestors Nazi salute in Vancouver ...,1.0,0.0


In [12]:
#Posts below a score of 100 will be dropped
AHS = AHS.sort_values(by = "score", ascending = False)
AHSNew = AHS[(AHS['score'] > 5) & (AHS['num_comments'] > 5)]
AHSNew['selftext'] = AHSNew['selftext'].fillna(" ")
AHSNew

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,selftext,title,score,num_comments
7380,,The ACTUAL FUCKING NAZIS AND WHITE NATIONALIST...,43178.0,2141.0
14743,[deleted],"The_Donald is a violent, extremist group being...",38130.0,2071.0
6231,,HanAssholeSolo wished for people to be doxxed ...,30570.0,1710.0
10093,"For those who may not be aware, /r/The_donald,...",Reddit admins have confirmed they are comforta...,30338.0,2017.0
5426,,/r/The_Donald Has Built A Document With The Ad...,28866.0,2375.0
...,...,...,...,...
564,,"""Tactics used by cancerous elements to stop fr...",6.0,9.0
16658,,"Moderators at r/azula refuse to remove porn ""a...",6.0,11.0
523,,On African Americans threatening to move to Ge...,6.0,6.0
1167,,"""Time to remove [Muslims] by any means people....",6.0,13.0


#####Analysing the subreddit to extract names of hateful subreddits from SubredditDrama

In [13]:
SD = pd.read_csv("/content/gdrive/MyDrive/SubredditDramaText.csv")
SD

,selftext,title,score,num_comments
0,NaN,Should someone kill themselves if they have a ...,37.0,97.0
1,NaN,2XC debates with OP as to whether a women only...,158.0,447.0
2,NaN,/r/occult is arguing about something. I have n...,1274.0,473.0
3,NaN,Refugee Drama in r/Islam,10.0,56.0
4,NaN,/u/bestofoutrageculture stickies a post to the...,248.0,209.0
...,...,...,...,...
70584,[deleted],r/AskTeenGirls mod having a fit over guys answ...,3.0,8.0
70585,[removed],r/Dota2 Mods turn off comments after Chinese c...,1.0,4.0
70586,[removed],FYI r/atheism is handing out lifetime bans for...,1.0,5.0
70587,[removed],R/History Perma Banned for asking about Moors,0.0,28.0


In [14]:
#Posts below a score of 100 will be dropped
SD = SD.sort_values(by = "score", ascending = False)
SDNew = SD[(SD['score'] > 400) & (SD['num_comments'] > 200)]
SDNew['selftext'] = SDNew['selftext'].fillna(" ")
SDNew

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,selftext,title,score,num_comments
68129,/r/The_Donald has been quarantined. Discuss th...,/r/The_Donald has been quarantined. Discuss th...,65481.0,24078.0
60526,,r/fuckthealtright mod made a detailed post of ...,51349.0,3311.0
53887,"It's literally the ENTIRE comment section, but...","r/The_Donald is imploding, following Trump's p...",41074.0,6525.0
44455,The_Donald has [gone private](/r/the_donald). ...,The_donald has gone private in protest of thei...,39328.0,10391.0
41072,/r/altright has been banned by the admins for ...,/r/altright has been banned. Discuss this here!,34168.0,8274.0
...,...,...,...,...
66666,,Is it alright for a female jew to write contov...,401.0,259.0
25407,,"Admins ask /r/guns to remove sidebar picture, ...",401.0,671.0
54811,https://www.reddit.com/r/worstof/comments/8209...,Users in /r/WorstOf try to figure out what dox...,401.0,389.0
41443,,Nicki Minaj's ex makes homeless man do pushups...,401.0,259.0


#####FIinding reddit name matches from the text

In [46]:
def clean_string(st):
  st = st.lower()
  st = st.replace(",","")
  st = st.replace(".","")
  st = st.replace(":","")
  st = st.replace("!","")
  st = st.replace("'","")
  st = st.replace(")","")
  st = st.replace("(","")
  st = st.replace("[","")
  st = st.replace("]","")
  st = st.replace("https","")
  return st


def find_matches(text):
  matches = {}
  for i in range(0, len(text)):
    m = []
    x = text.iloc[i]

    y = re.findall(r"/r/([^\s/]+)", x['title'])
    if len(y) > 0:
      for i in y:
        temp = clean_string(i)
        m.append(temp)

    z = re.findall(r"/r/([^\s/]+)", x['selftext'])
    if len(z) > 0:
      for i in z:
        temp = clean_string(i)
        m.append(temp)

    for i in m:
      if i not in matches.keys():
        matches[i] = 1
      else:
        matches[i] = matches[i] + 1
      break
  
  print("Matches found:", len(matches))
  print(matches)
  print("\n\n")

  temp_db = []
  for i in DB:
    temp_db.append(i.lower())

  matcheswithdb = []
  for i in matches.keys():
    if i in temp_db:
      matcheswithdb.append(i)

  print("Matches also found in quarantined/banned subreddits:", len(matcheswithdb))
  print(len(matcheswithdb), matcheswithdb)
  print("\n\n")

  print("Removing matches found in DB")
  for i in matcheswithdb:
    matches.pop(i)

  print("Final matches:", len(matches))
  print(matches)
  return matches

In [47]:
AHS_Final = find_matches(AHSNew)

Matches found: 537
{'the_donald': 388, 'physical_removal': 39, 'altright': 80, 'incels': 59, 'imgoingtohellforthis': 16, 'uncensorednews': 99, 'onguardforthee': 3, 'whiterights': 37, 'waterniggas': 1, 'blog': 1, 'greatawakening': 10, 'milliondollarextreme': 55, 'all': 9, 'conservative': 30, 'subredditdrama': 1, 'honkler': 4, 'race_realism': 1, 'obese_niggers': 1, 'topmindsofreddit': 2, 'shitpoliticssays': 6, 'metacanada': 72, 'conspiracy': 126, 'kiachatroom': 1, 'landofbuddies': 1, 'offensivespeech': 4, 'chadfish': 2, 'politics': 8, 'cringanarchy': 1, 'games': 1, 'canada': 22, 'stonetoss': 2, 'majorityrights': 1, 'drama': 13, 'antifa': 4, 'sjwhate': 10, 'clownworldwar2': 1, 'askthedonald': 1, 'racialrealism': 1, 'hillaryforprison': 4, 'catholicism': 11, 'israel': 18, 'rightwinglgbt': 6, 'clownworldwar': 8, 'blackcrime': 2, 'cringeanarchy': 73, 'modnews': 1, 'justiceserved': 2, 'transgenderkids': 1, 'the_donaldunleashed': 3, 'debatefascism': 3, 'againsthatesubreddits': 18, 'incest': 1, 

In [48]:
SD_Final = find_matches(SDNew)

Matches found: 714
{'the_donald': 45, 'altright': 3, 'trumpgret': 2, 'videos': 22, 'theredpill': 5, 'watchpeopledie': 2, 'incels': 2, 'physical_removal': 2, 'nfl': 8, 'teenagers': 3, 'gaming': 16, 'smashbros': 4, 'chapotraphouse': 7, 'conspiracy': 48, 'frenworld': 1, 'worldnews': 14, 'asktrumpsupporters': 2, 'politics**': 1, 'uncensorednews': 5, 'leagueoflegends': 9, 'battlefield': 6, 'wikileaks': 2, 'unexplainedphotos': 1, 'makenewfriendshere': 1, 'games': 15, 'amitheasshole': 7, 'kotakuinaction': 41, 'cringeanarchy': 9, 'rickandmorty': 8, 'iamatotalpieceofshit': 4, 'joerogan': 2, 'pizzagate': 1, 'shoplifting': 2, 'television': 8, 'pics': 37, 'pcgaming': 14, 'me_irl': 3, 'unpopularopinion': 5, 'deepfakes': 1, 'greatawakening': 6, 'hearthstone': 6, 'rainbow6': 3, 'fitgirls': 2, 'freefolk': 3, 'livestreamfail': 11, 'marchagainsttrump': 1, 'socialism': 6, 'memes': 1, 'goldtesting': 1, 'morbidquestions': 6, 'iama': 29, 'stopgaming': 1, 'coachella': 1, 'ice_poseidon2': 2, 'oculus': 3, 'sec

In [51]:
#sorting based on score
ahs = (sorted(AHS_Final.items(), key = lambda kv:(kv[1], kv[0]), reverse = True))
sd = (sorted(SD_Final.items(), key = lambda kv:(kv[1], kv[0]), reverse = True))

#####Extracting top hateful subreddits

In [55]:
final = []

#finding common ground
common_names = []
for i in sd:
  for j in ahs:
    if i[0]  == j[0]:
      x = (i[0], i[1]+j[1])
      common_names.append(i[0])
      final.append(x)

#adding the rest
for i in sd:
  if i[0] not in common_names:
    final.append(i)

for i in ahs:
  if i[0] not in common_names:
    final.append(i)

print("Length", len(final))
print(final)

Length 1096
[('news', 71), ('conspiracy', 174), ('kotakuinaction', 63), ('pics', 39), ('legaladvice', 38), ('drama', 46), ('politics', 39), ('subredditdrama', 29), ('askreddit', 26), ('videos', 23), ('movies', 20), ('announcements', 24), ('fatpeoplehate', 19), ('gaming', 17), ('games', 16), ('christianity', 17), ('worldnews', 30), ('pcgaming', 15), ('livestreamfail', 14), ('conservative', 41), ('bitcoin', 12), ('television', 9), ('libertarian', 11), ('europe', 16), ('topmindsofreddit', 8), ('subredditoftheday', 8), ('socialism', 9), ('pussypassdenied', 8), ('morbidquestions', 7), ('jordanpeterson', 12), ('greatawakening', 16), ('blackpeopletwitter', 9), ('aww', 8), ('wallstreetbets', 6), ('unpopularopinion', 17), ('uncensorednews', 104), ('outoftheloop', 6), ('islam', 22), ('askhistorians', 6), ('wtf', 5), ('publicfreakout', 10), ('iamatotalpieceofshit', 5), ('european', 185), ('blog', 5), ('upliftingnews', 4), ('subredditcancer', 9), ('shitredditsays', 4), ('roastme', 4), ('protectand

In [56]:
final2 = []

#picking the reddits with more than 5 mentions
for i in final:
  if i[1] > 5:
    final2.append(i)

print(len(final2))
print(final2)

117
[('news', 71), ('conspiracy', 174), ('kotakuinaction', 63), ('pics', 39), ('legaladvice', 38), ('drama', 46), ('politics', 39), ('subredditdrama', 29), ('askreddit', 26), ('videos', 23), ('movies', 20), ('announcements', 24), ('fatpeoplehate', 19), ('gaming', 17), ('games', 16), ('christianity', 17), ('worldnews', 30), ('pcgaming', 15), ('livestreamfail', 14), ('conservative', 41), ('bitcoin', 12), ('television', 9), ('libertarian', 11), ('europe', 16), ('topmindsofreddit', 8), ('subredditoftheday', 8), ('socialism', 9), ('pussypassdenied', 8), ('morbidquestions', 7), ('jordanpeterson', 12), ('greatawakening', 16), ('blackpeopletwitter', 9), ('aww', 8), ('wallstreetbets', 6), ('unpopularopinion', 17), ('uncensorednews', 104), ('outoftheloop', 6), ('islam', 22), ('askhistorians', 6), ('publicfreakout', 10), ('european', 185), ('subredditcancer', 9), ('mensrights', 9), ('h3h3productions', 7), ('enoughtrumpspam', 8), ('all', 12), ('tumblrinaction', 6), ('pussypass', 26), ('metacanada'

In [57]:
DH = []
for i in final2:
  DH.append(i[0])
DH

['news',
 'conspiracy',
 'kotakuinaction',
 'pics',
 'legaladvice',
 'drama',
 'politics',
 'subredditdrama',
 'askreddit',
 'videos',
 'movies',
 'announcements',
 'fatpeoplehate',
 'gaming',
 'games',
 'christianity',
 'worldnews',
 'pcgaming',
 'livestreamfail',
 'conservative',
 'bitcoin',
 'television',
 'libertarian',
 'europe',
 'topmindsofreddit',
 'subredditoftheday',
 'socialism',
 'pussypassdenied',
 'morbidquestions',
 'jordanpeterson',
 'greatawakening',
 'blackpeopletwitter',
 'aww',
 'wallstreetbets',
 'unpopularopinion',
 'uncensorednews',
 'outoftheloop',
 'islam',
 'askhistorians',
 'publicfreakout',
 'european',
 'subredditcancer',
 'mensrights',
 'h3h3productions',
 'enoughtrumpspam',
 'all',
 'tumblrinaction',
 'pussypass',
 'metacanada',
 'latestagecapitalism',
 'imgoingtohellforthis',
 'canada',
 'asktrumpsupporters',
 'againsthatesubreddits',
 '4chan',
 'thenewright',
 'sjwhate',
 'milliondollarextreme',
 'gendercritical',
 'frenworld',
 'easternsunrising',
 'ca

#Hypotheses 3

H3. (a) User participation in hateful or dangerous subreddits negatively changes the nature of their participation within the broader community and (b) this can be corrected by community bans
and quarantines. These hypotheses, if valid, will show that
community bans and quarantines as pre-emptive moderator
actions can reduce offensiveness within the broader community. If H3(a) is invalid, we will have shown that user
behavior in outside communities is not influenced by their
participation in hateful or dangerous communities. If H3(b)
is invalid, we will have shown that there is a need for more
nuanced pre-emptive interventions to mitigate the impact
of hateful and dangerous communities.

###Offensive Speech Classifier

https://github.com/t-davidson/hate-speech-and-offensive-language

In [89]:
!pip2 install vaderSentiment
!pip2 install textstat
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
!python2 /content/gdrive/MyDrive/Classifier/classifier/classifier.py

Streaming output truncated to the last 5000 lines.
"@ChrisCJackson: @realDonaldTrump I went to Trump Scotland last week and a gopher sexually assaulted my wife. After you, bet she loved it!
Neither
"@Nate_StuLLeR: @realDonaldTrump what was your bank account lookin like at age 24?"  Not too great!
Neither
"@BoxyBate: @realDonaldTrump: Can I get a hello from a UK (England) fan?"  Hello!
Neither
@anitanewell   True!
Neither
"@Queenjess32: @realDonaldTrump you're awesome."  I like you toooo-thanks!
Neither
"@tcasson21: @realDonaldTrump craig ferguson called you a bad person the other day? what is your opinion of him?"  Who is Craig Ferguson?
Neither
"@JewelsInLotus: @realDonaldTrump @BG9half @TheScotsman Amazing Beautiful Place!!"  Thanks!
Neither
"@Robert_MacRae: Next time your in Scotland can we go knock down some windmills ? @realDonaldTrump"  and save the birds!
Neither
"@FraserPorteous2: @realDonaldTrump I took a drive to Trump Scotland last week- what an incredible course your built.

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Built TF-IDF array
Built POS array
Built other feature array
Running classification model...
accuracy 0.896017431304
